In [2]:
import pandas as pd
import numpy as np

In [32]:
file = '../DATA/titanic.csv'

In [33]:
tDF = pd.read_csv(file)

In [34]:
tDF.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 데이터 전처리

In [36]:
# 잘 모르겠는 컬럼 데이터 의미 파악
tDF['who'].unique()  ## who에는 어린이, 남자, 여자 존재
tDF['adult_male'].unique()   ## T -> male / F -> Female
tDF['pclass'].unique()  # 1,2,3 있음

array([3, 1, 2], dtype=int64)

In [8]:
tDF.info()


# null값이 많아서 deck 컬럼 삭제 -> 근데 deck 컬럼 데이터가 의미하는게 뭔지 몰라서 찝찝함
# sex 와 who 범주형으로 형변환해야 함
# 분석과 관련없는 컬럼 삭제 ( sibsp 컬럼과 parch 컬럼)
# pclass 컬럼은 class 컬럼과 중복되는 의미를 가지니 제거
# alive 컬럼은 survived 컬럼과 중복되는 의미의 데이터를 가지니 제거
# adult_male 컬럼은 sex 컬럼과 중복되는 의미의 데이터를 가지니 제거
# embarked 컬럼은 embark_town 컬럼과 중복되는 의미의 데이터를 가지니 둘 중 하나 제거
# survived 컬럼은 범주형으로 분류해도 되지만 생존자 수를 세아리는데 유용할거 같아 int타입으로 두겠음
# 나머지 컬럼에 존재하는 결측치는 다른 값으로 대체

#### survived가 1인 데이터를 생존자 데이터로 분류

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [39]:
# 중복 컬럼이나 관계 없는 컬럼 제거
tDF = tDF.drop(labels=['sibsp','parch','alive','embarked','deck','adult_male','pclass'], axis = 1)

In [10]:
# 범주형으로 형변환
tDF['sex'] = tDF['sex'].astype('category')
tDF['who'] = tDF['who'].astype('category')

In [12]:
# 결측치 처리
tDF.isna().sum()  ## age와 embark_town에 결측치 존재

survived         0
pclass           0
sex              0
age            177
fare             0
class            0
who              0
embark_town      2
alone            0
dtype: int64

In [13]:
# age에 결측치 대체값으로 중앙값을 넣어봄
tDF['age'].fillna(tDF['age'].median(), inplace=True)

# embark_town에 결측치 대체값으로 최빈값을 넣어봄
tDF['embark_town'].fillna(tDF['embark_town'].mode()[0], inplace=True)

In [14]:
# 결측치 제거된 것 확인
tDF.isna().sum()   # 0

survived       0
pclass         0
sex            0
age            0
fare           0
class          0
who            0
embark_town    0
alone          0
dtype: int64

In [15]:
# 생존자 수
tDF['survived'].sum()

342

In [16]:
# 생존자들의 데이터 
aliveDF = tDF[tDF['survived'] == 1]  

In [17]:
aliveDF.describe(include='all')

,survived,pclass,sex,age,fare,class,who,embark_town,alone
count,342.0,342.000000,342,342.000000,342.000000,342,342,342,342
unique,NaN,NaN,2,NaN,NaN,3,3,3,2
top,NaN,NaN,female,NaN,NaN,First,woman,Southampton,False
freq,NaN,NaN,233,NaN,NaN,136,205,219,179
mean,1.0,1.950292,NaN,28.291433,48.395408,NaN,NaN,NaN,NaN
std,0.0,0.863321,NaN,13.764425,66.596998,NaN,NaN,NaN,NaN
min,1.0,1.000000,NaN,0.420000,0.000000,NaN,NaN,NaN,NaN
25%,1.0,1.000000,NaN,21.000000,12.475000,NaN,NaN,NaN,NaN
50%,1.0,2.000000,NaN,28.000000,26.000000,NaN,NaN,NaN,NaN
75%,1.0,3.000000,NaN,35.000000,57.000000,NaN,NaN,NaN,NaN


### 성별과 생존의 상관성
- 상관성을 가진다.
- 여자가 남자보다 더 많은 비율로 생존했다.

In [18]:
aliveDF['sex'].value_counts()
# 생존자 중에서는 여자가 233명으로 남자보다 두배 이상 많이 생존했다.

sex
female    233
male      109
Name: count, dtype: int64

In [19]:
aliveDF['sex'].value_counts(normalize=True).round(2)*100
# 생존자의 수 중에서 여자의 총 비율은 68%, 남자는 32%이다.

sex
female    68.0
male      32.0
Name: proportion, dtype: float64

In [20]:
# 그럼 생존률을 봐보자
(aliveDF['sex'].value_counts()/tDF['sex'].value_counts()*100).round(0)
# 여자의 생존률은 74%
# 남자의 생존률은 19%
# 성별과 생존률은 유의미한 상관관계를 가지는 것 같다.

sex
female    74.0
male      19.0
Name: count, dtype: float64

In [41]:
(aliveDF['who'].value_counts()/tDF['who'].value_counts()*100).round(0).sort_values(ascending=False)
# 여자가 가장 많이 살아남았고, 그다음 남자, 아이 순이다.

who
woman    76.0
child    59.0
man      16.0
Name: count, dtype: float64

### 나이와 생존의 상관성
- 상관성을 가진다.
- 19~ 50세가 생존률이 가장 높다
- 너무 나이가 많거나 적은 경우 생존률이 적다. 
- 그러나 안겨서 대피할 수 있는 0~5세의 생존률이 5~12세 생존률보다 높다.

In [22]:
# 나이 구간을 설정해서 직관적으로 파악해보고자 함 
age = [0,5,12,19,50,80]
aliveDF['age'].value_counts(bins=age)
# 19 ~ 50 세가 가장 많다.

age
(19.0, 50.0]     241
(12.0, 19.0]      39
(-0.001, 5.0]     31
(50.0, 80.0]      22
(5.0, 12.0]        9
Name: count, dtype: int64

In [38]:
# 이것을 빈도로 봐보자

(aliveDF['age'].value_counts(bins=age, normalize=True)*100).round(0)
# 19~50세 생존자 : 70%  -> 전체의 70% 차지!!
# 12~19세 생존자 : 15%
# 0~5세 생존자 : 9%   -> 안겨서 대피가 가능했기에 5~12세보다 생존률이 높음
# 50~80세 생존자 : 6%
# 5~12세 생존자 : 3% -> 전체의 3%를 차지하며 가장 낮다.

age
(19.0, 50.0]     70.0
(12.0, 19.0]     11.0
(-0.001, 5.0]     9.0
(50.0, 80.0]      6.0
(5.0, 12.0]       3.0
Name: proportion, dtype: float64

### 등석과 생존의 상관성
- 상관성을 가진다.
- 등석이 높을수록 생존률이 높다

In [24]:
aliveDF['class'].value_counts()
# 생존자 수는 1등석 > 3등석 > 2등석 순이다.
# 비율이 중요하므로 생존률을 한번 봐보자 

class
First     136
Third     119
Second     87
Name: count, dtype: int64

In [25]:
(aliveDF['class'].value_counts() / tDF['class'].value_counts()*100).round(0)

# 생존률은 1등급 > 2등급 > 3등급
#          63%  >  47%  > 24%   로 상관관계가 있다고 파악된다.

class
First     63.0
Second    47.0
Third     24.0
Name: count, dtype: float64

### 혼자 있었던 것과 생존의 상관성
- 혼자 있었던 경우 생존률이 1.5배 이상 낮다.
- 궁금해서 해봤어요

In [28]:
(aliveDF['alone'].value_counts()/tDF['alone'].value_counts()*100).round(0)

# 혼자 없었던 경우의 생존률이 더 높다. 

alone
False    51.0
True     30.0
Name: count, dtype: float64

- 혼자 있었던 것과 성별 나이와 생존의 상관성

In [29]:
# + who에 따른 혼자 있었는데 생존한 생존자 수
aliveDF[(aliveDF['alone'] == True) & (aliveDF['who'] == 'man')].shape[0]  # 64명
aliveDF[(aliveDF['alone'] == True) & (aliveDF['who'] == 'woman')].shape[0]  # 95명
aliveDF[(aliveDF['alone'] == True) & (aliveDF['who'] == 'child')].value_counts()  # 4명
# 혼자 있던 아이 중에서 생존한 아이는 고작 4명이며 모두 여자이다.

survived  pclass  sex     age   fare     class  who    embark_town  alone
1         3       female  5.0   12.4750  Third  child  Southampton  True     1
                          13.0  7.2292   Third  child  Cherbourg    True     1
                          15.0  7.2250   Third  child  Cherbourg    True     1
                                8.0292   Third  child  Queenstown   True     1
Name: count, dtype: int64